In [1]:
import numpy as np
import pandas as pd
import openpyxl
import warnings
warnings.filterwarnings(action='ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import math

In [2]:
def roundup(x):
    i, f = divmod(x, 1)
    return int(i + ((f >= 0.5) if (x > 0) else (f > 0.5)))

In [3]:
df = pd.read_excel('Transfer Order by Product6_21_2022 5 08 40 PM.xlsx', header=4)
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')

In [4]:
ph=df[df.country=='Philippines']

In [5]:
ph.sales_person_name.unique()

array(['Darlene Dela Fuente', 'Carlo Molina', 'Jonathan Familar',
       'Randy Santos'], dtype=object)

In [6]:
nm = ph.sales_person_name.unique()[1]

In [7]:
# transfer order

ph_sales = ph[ph.sales_person_name==nm]
ph_sales.product_code = [' '.join(i.split()) for i in ph_sales.product_code]

In [8]:
df1 = pd.read_excel('incentive_ph.xlsx', sheet_name='strategy master 2')
df1.columns=df1.columns.str.lower()
df1.columns=df1.columns.str.replace(' ','_')

In [9]:
df1.product_code=df1.product_code.astype(str)
df1.product_code=[' '.join(i.split()) for i in df1.product_code]

In [10]:
# strategy master

# ph_sm=df1[df1.country=='Philippines']
ph_sm=df1.copy()
ph_sm.product_code=ph_sm.product_code.astype(str)
ph_sm.sub_category=ph_sm.sub_category.str.lower()

In [11]:
# incentive calculation

df2 = pd.read_excel('incentive_ph.xlsx', sheet_name='results')
df2.sub_cat = df2.sub_cat.str.lower()

In [12]:
# attached sub_category to transfer order

ph_sales['sub_category']=np.nan
for i, p in zip(ph_sales.index, ph_sales.product_code):
    for pc, sc in zip(ph_sm.product_code, ph_sm.sub_category):
        if p==pc:
            ph_sales.loc[i,'sub_category']=sc

In [13]:
# groupby sub_category for transfer order

ph_sales1 = ph_sales.dropna(subset=['sub_category'])
sales_gp = ph_sales1.groupby(['sales_person_name','sub_category']).agg(qty=('qty','sum')).reset_index()

In [15]:
# attached qty to results template df2

df2['qty']=0
df2['sales_person_name']=np.nan
for i, s in zip(df2.index, df2.sub_cat):
    for sc, q, sn in zip(sales_gp.sub_category, sales_gp.qty, sales_gp.sales_person_name):
        if s==sc:
            df2.loc[i,'qty']=q
            df2.loc[i,'sales_person_name']=sn
        else:
            df2.loc[i,'sales_person_name']=sn

In [17]:
for i,q,t in zip(df2.index, df2.qty, df2.target):
    if t<=0:
        df2.loc[i,'achieve']=0
    else:
        d=q/t*100
        df2.loc[i,'achieve']=roundup(d)


In [18]:
df2[nm]=0

for i, w, j in zip(df2.index, df2.weightage,df2.achieve):
    if j<=65:
        df2.loc[i,nm]=0
        
    elif j > 65 and j <=100:
        x = j*w
        df2.loc[i,nm]=roundup(x)
        
    elif j > 100 and j <= 104:
        j=110
        x = j*w
        df2.loc[i,nm]=roundup(x)
        
    elif j > 105 and j <= 109:
        j=120
        x=j*w
        df2.loc[i,nm]=roundup(x)
        
    elif j > 110 and j <=119:
        j=130
        x=j*w
        df2.loc[i,nm]=roundup(x)
        
    else:
        j=140
        x=j*w
        df2.loc[i,nm]=roundup(x)


df2.loc[df2.last_valid_index()+1,nm]=df2[nm].sum()
df2.loc[df2.last_valid_index(),'sub_cat']='Total Reward'

In [19]:
df3=df2[['sub_cat',nm]]

In [22]:
df3

,sub_cat,Carlo Molina
0,everx posterior,0.0
1,everx flow,6.0
2,g-aenial,0.0
3,g-aenial universal flo,0.0
4,g-aenial universal injectable,4.0
5,modeling liquid,0.0
6,sticktech,0.0
7,g-premio bond (incl. dca),6.0
8,g-cem capsule,0.0
9,g-cem linkforce,0.0
